## Сравнение КССС и WAY

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

class DataComparator:
    def __init__(self):
        self.results = {}
        self.statistics = {}
        
    def load_data(self, master_file: str, receiver_file: str, 
                  master_sheets: List[str] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Загрузка данных из Excel файлов
        """
        print("Загрузка данных...")
        
        # Загрузка данных из системы-получателя
        receiver_df = pd.read_excel(receiver_file)
        print(f"Загружено {len(receiver_df)} записей из системы-получателя")
        
        # Загрузка данных из мастер-системы (может быть на нескольких листах)
        if master_sheets:
            master_dfs = []
            for sheet in master_sheets:
                df = pd.read_excel(master_file, sheet_name=sheet)
                master_dfs.append(df)
                print(f"Загружено {len(df)} записей с листа '{sheet}'")
            master_df = pd.concat(master_dfs, ignore_index=True)
        else:
            master_df = pd.read_excel(master_file)
            print(f"Загружено {len(master_df)} записей из мастер-системы")
            
        return master_df, receiver_df
    
    def find_matching_keys(self, master_df: pd.DataFrame, receiver_df: pd.DataFrame,
                          master_key_col: str, receiver_key_col: str) -> Dict:
        """
        Поиск ключей из системы-получателя в мастер-системе
        """
        print("\nПоиск совпадений ключей...")
        
        master_keys = set(master_df[master_key_col].dropna().astype(str))
        receiver_keys = set(receiver_df[receiver_key_col].dropna().astype(str))
        
        found_keys = receiver_keys.intersection(master_keys)
        missing_keys = receiver_keys - master_keys
        
        key_stats = {
            'total_receiver_keys': len(receiver_keys),
            'found_keys': len(found_keys),
            'missing_keys': len(missing_keys),
            'coverage_percentage': (len(found_keys) / len(receiver_keys)) * 100 if receiver_keys else 0
        }
        
        print(f"Всего ключей в системе-получателе: {key_stats['total_receiver_keys']}")
        print(f"Найдено в мастер-системе: {key_stats['found_keys']}")
        print(f"Не найдено: {key_stats['missing_keys']}")
        print(f"Покрытие: {key_stats['coverage_percentage']:.2f}%")
        
        return {
            'stats': key_stats,
            'found_keys': list(found_keys),
            'missing_keys': list(missing_keys)
        }
    
    def compare_columns(self, master_df: pd.DataFrame, receiver_df: pd.DataFrame,
                       found_keys: List, master_key_col: str, receiver_key_col: str,
                       master_columns: List[str], receiver_columns: List[str]) -> pd.DataFrame:
        """
        Сравнение значений колонок для найденных ключей
        """
        print("\nСравнение значений колонок...")
        
        if len(master_columns) != len(receiver_columns):
            raise ValueError("Количество колонок для сравнения должно быть одинаковым")
        
        # Фильтруем данные только по найденным ключам
        receiver_filtered = receiver_df[receiver_df[receiver_key_col].astype(str).isin(found_keys)].copy()
        master_filtered = master_df[master_df[master_key_col].astype(str).isin(found_keys)].copy()
        
        # Создаем результат сравнения
        comparison_results = receiver_filtered.copy()
        
        # Добавляем колонки для результатов сравнения
        column_stats = {}
        
        for i, (master_col, receiver_col) in enumerate(zip(master_columns, receiver_columns)):
            comparison_col = f'Сравнение_{receiver_col}'
            status_col = f'Статус_{receiver_col}'
            
            # Создаем словарь для быстрого поиска значений из мастер-системы
            master_dict = dict(zip(
                master_filtered[master_key_col].astype(str), 
                master_filtered[master_col].astype(str)
            ))
            
            # Сравниваем значения
            comparison_results[comparison_col] = comparison_results[receiver_key_col].astype(str).map(master_dict)
            comparison_results[status_col] = np.where(
                comparison_results[receiver_col].astype(str) == comparison_results[comparison_col],
                'Совпадает',
                'Не совпадает'
            )
            
            # Статистика по колонке
            matches = (comparison_results[status_col] == 'Совпадает').sum()
            total = len(comparison_results)
            match_percentage = (matches / total) * 100 if total > 0 else 0
            
            column_stats[receiver_col] = {
                'total_comparisons': total,
                'matches': matches,
                'mismatches': total - matches,
                'match_percentage': match_percentage
            }
            
            print(f"Колонка '{receiver_col}': {matches}/{total} совпадений ({match_percentage:.2f}%)")
        
        self.results['comparison'] = comparison_results
        self.results['column_stats'] = column_stats
        
        return comparison_results
    
    def generate_statistics(self, key_results: Dict, column_stats: Dict) -> Dict:
        """
        Генерация общей статистики
        """
        total_stats = {
            'key_comparison': key_results['stats'],
            'column_comparison': column_stats,
            'overall_match_percentage': np.mean([stats['match_percentage'] for stats in column_stats.values()])
        }
        
        self.statistics = total_stats
        return total_stats
    
    def save_results(self, output_file: str, comparison_results: pd.DataFrame, 
                    key_results: Dict, statistics: Dict):
        """
        Сохранение результатов в Excel файл
        """
        print(f"\nСохранение результатов в {output_file}...")
        
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # Лист с результатами сравнения
            comparison_results.to_excel(writer, sheet_name='Результаты_сравнения', index=False)
            
            # Лист со статистикой ключей
            key_stats_df = pd.DataFrame([statistics['key_comparison']])
            key_stats_df.to_excel(writer, sheet_name='Статистика_ключей', index=False)
            
            # Лист со статистикой колонок
            column_stats_list = []
            for col_name, stats in statistics['column_comparison'].items():
                stats_row = {'Колонка': col_name, **stats}
                column_stats_list.append(stats_row)
            
            column_stats_df = pd.DataFrame(column_stats_list)
            column_stats_df.to_excel(writer, sheet_name='Статистика_колонок', index=False)
            
            # Лист с отсутствующими ключами
            missing_keys_df = pd.DataFrame({'Отсутствующие_ключи': key_results['missing_keys']})
            missing_keys_df.to_excel(writer, sheet_name='Отсутствующие_ключи', index=False)
        
        print("Результаты успешно сохранены!")

def main_comparison():
    """
    Основная функция для выполнения сравнения
    """
    # НАСТРОЙКИ - ЗАПОЛНИТЕ ЭТИ ПАРАМЕТРЫ
    # ====================================
    
    # Пути к файлам
    MASTER_FILE = "master_system.xlsx"  # Файл мастер-системы
    RECEIVER_FILE = "receiver_system.xlsx"  # Файл системы-получателя
    OUTPUT_FILE = "comparison_results.xlsx"  # Файл для результатов
    
    # Названия листов в файле мастер-системы (если данных больше миллиона)
    MASTER_SHEETS = ["Лист1", "Лист2"]  # Если один лист - оставить пустым []
    
    # Названия колонок с ключами
    MASTER_KEY_COLUMN = "ID"  # Колонка с ключом в мастер-системе
    RECEIVER_KEY_COLUMN = "Ключ"  # Колонка с ключом в системе-получателе
    
    # Колонки для сравнения (должны быть в том же порядке для обеих систем)
    MASTER_COMPARE_COLUMNS = ["Название", "Количество", "Цена"]  # Колонки из мастер-системы
    RECEIVER_COMPARE_COLUMNS = ["Наименование", "Кол_во", "Стоимость"]  # Соответствующие колонки из системы-получателя
    
    # ====================================
    
    # Создаем компаратор
    comparator = DataComparator()
    
    try:
        # Загружаем данные
        master_df, receiver_df = comparator.load_data(
            MASTER_FILE, RECEIVER_FILE, MASTER_SHEETS
        )
        
        # Поиск совпадений ключей
        key_results = comparator.find_matching_keys(
            master_df, receiver_df, MASTER_KEY_COLUMN, RECEIVER_KEY_COLUMN
        )
        
        # Сравнение колонок
        comparison_results = comparator.compare_columns(
            master_df, receiver_df, key_results['found_keys'],
            MASTER_KEY_COLUMN, RECEIVER_KEY_COLUMN,
            MASTER_COMPARE_COLUMNS, RECEIVER_COMPARE_COLUMNS
        )
        
        # Генерация статистики
        statistics = comparator.generate_statistics(key_results, comparator.results['column_stats'])
        
        # Сохранение результатов
        comparator.save_results(OUTPUT_FILE, comparison_results, key_results, statistics)
        
        # Вывод итоговой статистики
        print("\n" + "="*50)
        print("ИТОГОВАЯ СТАТИСТИКА")
        print("="*50)
        print(f"Общее покрытие ключей: {statistics['key_comparison']['coverage_percentage']:.2f}%")
        print(f"Средний процент совпадения колонок: {statistics['overall_match_percentage']:.2f}%")
        
        return comparator
        
    except Exception as e:
        print(f"Ошибка при выполнении сравнения: {e}")
        return None

# Запуск сравнения
comparator = main_comparison()


## Сравнение WAY и 1С

In [1]:
import pandas as pd
from typing import Dict, List

class CatalogComparator:
    def __init__(self, 
                 df1_fields: Dict[str, str] = None,
                 df2_fields: Dict[str, str] = None):
        """
        Инициализация компаратора с настройками полей
        
        Parameters:
        df1_fields - словарь с названиями полей для df1: {'code': 'код', 'name': 'наименование'}
        df2_fields - словарь с названиями полей для df2: {'code': 'код', 'name': 'наименование', 'full_name': 'Полное наименование'}
        """
        self.df1_fields = df1_fields or {'code': 'код', 'name': 'наименование'}
        self.df2_fields = df2_fields or {'code': 'код', 'name': 'наименование', 'full_name': 'Полное наименование'}
    
    def compare(self, df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
        """Основной метод сравнения"""
        
        # Подготавливаем датафреймы
        df1_prepared = self._prepare_dataframe(df1, self.df1_fields, 'df1')
        df2_prepared = self._prepare_dataframe(df2, self.df2_fields, 'df2')
        
        result_list = []
        
        # Обрабатываем каждую строку из первой выгрузки
        for idx, df1_row in df1_prepared.iterrows():
            code = df1_row['code']
            name_df1 = df1_row['name']
            
            # Ищем соответствия во второй выгрузке
            df2_matches = df2_prepared[df2_prepared['code'] == code]
            
            if df2_matches.empty:
                # Код не найден
                result_list.append(self._create_result_row(
                    code, name_df1, 'не найдено в выгрузке 2'
                ))
            else:
                # Проверяем совпадения по наименованиям
                match_found, match_info = self._check_name_matches(name_df1, df2_matches)
                
                if match_found:
                    result_list.append(self._create_result_row(
                        code, name_df1, 'совпадение найдено', 
                        match_info['name'], match_info['full_name']
                    ))
                else:
                    # Берем первую найденную запись для отображения
                    first_match = df2_matches.iloc[0]
                    result_list.append(self._create_result_row(
                        code, name_df1, 'нет совпадений по "наименование", "Полное наименование"',
                        first_match['name'], first_match['full_name']
                    ))
        
        return pd.DataFrame(result_list)
    
    def _prepare_dataframe(self, df: pd.DataFrame, field_mapping: Dict, df_name: str) -> pd.DataFrame:
        """Подготавливает датафрейм: переименовывает колонки и проверяет наличие полей"""
        required_fields = list(field_mapping.keys())
        
        # Проверяем наличие всех необходимых полей
        missing_fields = []
        for field in required_fields:
            if field_mapping[field] not in df.columns:
                missing_fields.append(field_mapping[field])
        
        if missing_fields:
            raise ValueError(f"В {df_name} отсутствуют поля: {missing_fields}")
        
        # Переименовываем колонки
        reverse_mapping = {v: k for k, v in field_mapping.items()}
        return df.rename(columns=reverse_mapping)[required_fields]
    
    def _check_name_matches(self, name_df1: str, df2_matches: pd.DataFrame) -> tuple:
        """Проверяет совпадения по наименованиям"""
        for _, df2_row in df2_matches.iterrows():
            # Проверяем обычное наименование
            if name_df1 == df2_row['name']:
                return True, {'name': df2_row['name'], 'full_name': df2_row['full_name']}
            
            # Проверяем полное наименование
            if name_df1 == df2_row['full_name']:
                return True, {'name': df2_row['name'], 'full_name': df2_row['full_name']}
        
        return False, {}
    
    def _create_result_row(self, code: str, name_df1: str, result: str, 
                          name_df2: str = None, full_name_df2: str = None) -> Dict:
        """Создает строку результата"""
        return {
            'код': code,
            'наименование_из_выгрузки_1': name_df1,
            'результат_сравнения': result,
            'наименование_из_выгрузки_2': name_df2,
            'полное_наименование_из_выгрузки_2': full_name_df2
        }




In [5]:
df2_custom = pd.read_excel("WAY.xlsx")
df1_custom = pd.read_excel("1C.xlsx")

In [3]:
df1_custom.head()

,КодКССС,Наименование
0,1,Мойка Стандарт - 2 класс
1,1,Бензин автомобильный Супер-98
2,1,Бензин автомобильный Премиум-95
3,100429,"Хол/чай Nestea лесные ягоды ПЭТ 0,5л"
4,100429,Бакалея


In [6]:
# Пример использования с кастомными названиями полей
if __name__ == "__main__":
    # Тестовые данные с разными названиями полей
    data1 = {
        'ID': ['001', '002', '003', '001', '004'],
        'Name': ['Товар А', 'Товар Б', 'Товар В', 'Товар А2', 'Товар Г']
    }
    
    data2 = {
        'ID': ['001', '002', '003', '001', '005'],
        'ShortName': ['Товар А', 'Товар Б2', 'Товар В', 'Товар А разный', 'Товар Д'],
        'FullName': ['Товар А полное', 'Товар Б', 'Товар В полное', 'Товар А2', 'Товар Д полное']
    }
    
    #df1_custom = pd.DataFrame(data1)
    #df2_custom = pd.DataFrame(data2)
    
    print("Выгрузка 1 с кастомными полями:")
    print(df1_custom)
    print("\nВыгрузка 2 с кастомными полями:")
    print(df2_custom)
    
    # Настраиваем маппинг полей
    df1_fields = {'code': 'КодКССС', 'name': 'Наименование'}
    df2_fields = {'code': 'CSCD_ID', 'name': 'NAME', 'full_name': 'F_NAME'}
    
    # Создаем компаратор с кастомными полями
    comparator = CatalogComparator(df1_fields, df2_fields)
    
    # Выполняем сравнение
    result_custom = comparator.compare(df1_custom, df2_custom)
    
    print("\nРезультат сравнения с кастомными полями:")
    print(result_custom)
    
    # Сохраняем в Excel
    result_custom.to_excel('результат_сравнения_кастомные_поля.xlsx', index=False)
    print("\nРезультат сохранен в файл 'результат_сравнения_кастомные_поля.xlsx'")

Выгрузка 1 с кастомными полями:
      КодКССС                                       Наименование
0           1                           Мойка Стандарт - 2 класс
1           1                      Бензин автомобильный Супер-98
2           1                    Бензин автомобильный Премиум-95
3      100429               Хол/чай Nestea лесные ягоды ПЭТ 0,5л
4      100429                                            Бакалея
...       ...                                                ...
8451  5002675                                       ВТОРЫЕ БЛЮДА
8452  5003037  Пирожное Kinder delice бисквитное с молочной н...
8453  5003037  Пирожное Kinder delice бисквитное с молочной н...
8454  5004856  Чипсы Lorenz Naturals картофельные с паприкой ...
8455  5004856  Чипсы Lorenz Naturals картофельные с паприкой ...

[8456 rows x 2 columns]

Выгрузка 2 с кастомными полями:
        CSCD_ID                                            NAME  \
0             0                                           Товар